# Case Study — Student Template
_Generated 2025-09-04_

This notebook shows the **expected workflow** and **deliverables** for your steady-state case study.
You will:
1) Load the **baseline** MODFLOW-2005 model (Flopy)
2) Define and apply your **scenario** from `case_student.yaml`
3) Run **baseline** and **scenario** (steady)
4) Produce **standard outputs**: head map, drawdown, observation heads, compact budget
5) Perform an **analytical check** (e.g., Thiem)
6) Write short **interpretation** and **sensitivity** notes


## 0. Environment
If needed, install packages (uncomment in a terminal or a cell):
```bash
# %pip install flopy pyyaml matplotlib numpy
```


In [ ]:
import os, pprint
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import yaml
import flopy
from flopy.utils import HeadFile, CellBudgetFile

plt.rcParams['figure.figsize'] = (8, 6)
def ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

def load_yaml(path):
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

def recarray_from_wells(wells):
    dtype = [('k', int), ('i', int), ('j', int), ('flux', float)]
    arr = np.zeros((len(wells),), dtype=dtype)
    for idx, w in enumerate(wells):
        arr[idx] = (w['layer'], w['row'], w['col'], w['rate'])
    return arr

def summarize_budget(cbc_path, terms, kstpkper=(0,0)):
    cbc = CellBudgetFile(cbc_path)
    out = {}
    for t in terms:
        try:
            data = cbc.get_data(text=t, kstpkper=kstpkper)
            if data:
                out[t] = float(np.sum(data[0]))
            else:
                out[t] = None
        except Exception:
            out[t] = None
    return out

def sample_heads(hds_path, lrc_list, kstpkper=None):
    hf = HeadFile(hds_path)
    arr = hf.get_data(kstpkper=kstpkper) if kstpkper is not None else hf.get_data()[-1]
    samples = []
    for (k,i,j) in lrc_list:
        samples.append({'k':k, 'i':i, 'j':j, 'head': float(arr[k, i, j])})
    return samples

def thiem_confined_drawdown(Q_abs_m3d, T_m2d, r_obs_m, r_ref_m):
    # s = (Q / (2πT)) * ln(r_obs/r_ref); Q_abs_m3d is positive magnitude
    return (Q_abs_m3d / (2.0 * np.pi * T_m2d)) * np.log(r_obs_m / r_ref_m)

print('Imports OK')


## 1. Load your scenario from YAML
Edit `case_student.yaml` (in the same folder) to define your case. Replace TODOs.


In [ ]:
CASE_YAML = 'case_student.yaml'
cfg = load_yaml(CASE_YAML)
pprint.pprint(cfg)

# basic checks (fail early if TODOs left)
assert 'csXX' not in cfg['name'], 'Please set a proper case id in yaml.'
assert '<your_case_id>' not in cfg['output']['workspace'], 'Please set output.workspace in yaml.'
assert '<your_case_id>' not in cfg['output']['modelname_suffix'], 'Please set output.modelname_suffix in yaml.'

base_ws   = cfg['model']['workspace']
namefile  = cfg['model']['namefile']
out_ws    = cfg['output']['workspace']
suffix    = cfg['output'].get('modelname_suffix', '')
ensure_dir(out_ws)


## 2. Load baseline model and duplicate to case workspace
We will run the **baseline** once (no changes), then your **scenario**.


In [ ]:
m_base = flopy.modflow.Modflow.load(namefile=namefile, model_ws=base_ws,
                                    forgive=True, check=False, exe_name='mf2005')
assert m_base is not None, 'Failed to load base model — check model.workspace and namefile in YAML.'
print('Loaded base model:', m_base.name)

m = m_base  # work on a clone moved to output
if suffix:
    m.name = f"{m.name}_{suffix}"
m.change_model_ws(out_ws)
print('Case model name:', m.name)
print('Workspace     :', m.model_ws)


## 3. Solver and steady settings
We keep a steady run; `NSTP=1`, `TSMULT=1`. Adjust solver tolerances and damping per YAML.


In [ ]:
solv = cfg['settings'].get('solver', {})
pcg = m.get_package('PCG')
if pcg is None:
    pcg = flopy.modflow.ModflowPcg(m)
pcg.mxiter = int(solv.get('mxiter', pcg.mxiter))
pcg.iter1  = int(solv.get('iter1',  pcg.iter1))
pcg.hclose = float(solv.get('hclose', pcg.hclose))
pcg.rclose = float(solv.get('rclose', pcg.rclose))
pcg.damp   = float(solv.get('damp',   getattr(pcg, 'damp', 0.0)))
m.dis.steady = [True] * m.dis.nper
m.dis.nstp   = [1] * m.dis.nper
m.dis.tsmult = [1.0] * m.dis.nper
print('Steady periods:', m.dis.steady)


## 4. BASELINE run (no scenario edits yet)
This captures heads for later **drawdown** maps. If your baseline already has wells or rivers, that’s fine — the scenario will compare against whatever the baseline is.


In [ ]:
m.write_input()
success_base, buff = m.run_model(report=True)
print('Baseline success:', success_base)
if not success_base:
    print('\n'.join(buff[-60:]))

hds_base_path = os.path.join(m.model_ws, m.name + '.hds')
hf_base = HeadFile(hds_base_path)
H_base = hf_base.get_data()[-1]
print('Baseline heads loaded:', H_base.shape)


## 5. Scenario edits (from YAML): wells, recharge patches, river tweaks, parameter multipliers
Keep scenario small and clearly documented.


In [ ]:
# Wells
wel_cfg = cfg['stresses'].get('wells', {})
wel = m.get_package('WEL')
if wel_cfg and wel_cfg.get('entries'):
    entries = wel_cfg['entries']
    mode = wel_cfg.get('mode', 'replace').lower()
    arr = recarray_from_wells(entries)
    if wel is None:
        wel = flopy.modflow.ModflowWel(m, stress_period_data={0: arr})
        print('Created WEL with', len(entries), 'entries')
    else:
        if mode == 'append':
            old = wel.stress_period_data
            base = old[0] if 0 in old else list(old.data.values())[0]
            merged = np.concatenate([base, arr])
            wel.stress_period_data = {0: merged}
            print('Appended wells; total entries:', len(merged))
        else:
            wel.stress_period_data = {0: arr}
            print('Replaced wells with', len(entries), 'entries')
else:
    print('No new wells specified.')

# Recharge
rch_cfg = cfg['stresses'].get('recharge', {})
rch = m.get_package('RCH')
if rch is not None and rch_cfg:
    base_r = rch.rech.array
    if isinstance(base_r, list):
        base_r = base_r[0]
    base_r = np.array(base_r, dtype=float)
    scale = float(rch_cfg.get('scale', 1.0))
    base_r *= scale
    # patches (optional)
    for p in rch_cfg.get('patches', []):
        if p.get('kind','rect') == 'rect':
            r0,r1 = int(p['row_min']), int(p['row_max'])
            c0,c1 = int(p['col_min']), int(p['col_max'])
            base_r[r0:r1+1, c0:c1+1] *= float(p.get('factor', 1.0))
    rch.rech = base_r
    print('Recharge edited. New mean R:', float(np.mean(base_r)))
else:
    print('No recharge edits.')

# Rivers
riv_cfg = cfg['stresses'].get('rivers', {})
riv = m.get_package('RIV')
if riv is not None and riv_cfg:
    spd = riv.stress_period_data
    key0 = 0 if 0 in spd else list(spd.data.keys())[0]
    arr = spd[key0].copy()
    if 'stage_offset' in riv_cfg:
        arr['stage'] = arr['stage'] + float(riv_cfg['stage_offset'])
    if 'conductance_factor' in riv_cfg:
        arr['cond']  = arr['cond']  * float(riv_cfg['conductance_factor'])
    riv.stress_period_data = {0: arr}
    print('RIV edits applied')
else:
    print('No river edits.')

# Parameter multipliers
par = cfg.get('parameters', {})
lpf = m.get_package('LPF')
if lpf is not None:
    if 'hk_factor' in par and hasattr(lpf, 'hk'):
        lpf.hk = np.array(lpf.hk.array, dtype=float) * float(par['hk_factor'])
    if 'vk_factor' in par:
        if hasattr(lpf, 'vka') and lpf.vka is not None:
            lpf.vka = np.array(lpf.vka.array, dtype=float) * float(par['vk_factor'])
        elif hasattr(lpf, 'vk') and lpf.vk is not None:
            lpf.vk  = np.array(lpf.vk.array,  dtype=float) * float(par['vk_factor'])
print('Scenario edits complete.')


## 6. Scenario run
Now write inputs and run the scenario in the case workspace.

In [ ]:
m.write_input()
success, buff = m.run_model(report=True)
print('Scenario success:', success)
if not success:
    print('\n'.join(buff[-60:]))

lst_path = os.path.join(m.model_ws, m.name + '.lst')
hds_path = os.path.join(m.model_ws, m.name + '.hds')
cbc_path = os.path.join(m.model_ws, m.name + '.cbc')
print('Outputs:', lst_path, hds_path, cbc_path)

hf = HeadFile(hds_path)
H = hf.get_data()[-1]
print('Scenario heads shape:', H.shape)


## 7. Standard outputs
Produce a head map (layer 1), a **drawdown map** (scenario − baseline), sampled heads, and a compact water budget. These are required in every report.

In [ ]:
# Head map (layer 1)
plt.figure()
plt.imshow(H[0], origin='lower')
plt.colorbar(label='Head (m)')
plt.title('Scenario Heads — Layer 1')
plt.xlabel('col'); plt.ylabel('row')
plt.show()

# Drawdown map vs baseline
DD = H[0] - H_base[0]
plt.figure()
plt.imshow(DD, origin='lower')
plt.colorbar(label='Drawdown (m)')
plt.title('Drawdown (Scenario − Baseline) — Layer 1')
plt.xlabel('col'); plt.ylabel('row')
plt.show()

# Head samples
lrc = cfg['observations'].get('heads_lrc', [])
samples_base = [{'k':k,'i':i,'j':j,'head': float(H_base[k,i,j])} for (k,i,j) in lrc]
samples_scen = [{'k':k,'i':i,'j':j,'head': float(H[k,i,j])} for (k,i,j) in lrc]
print('Observation heads (baseline vs scenario):')
for sb, ss in zip(samples_base, samples_scen):
    dh = ss['head'] - sb['head']
    print(f"(k,i,j)=({sb['k']},{sb['i']},{sb['j']})  H_base={sb['head']:.3f}  H_scen={ss['head']:.3f}  dH={dh:.3f}")

# Budget summary
terms = cfg['observations'].get('budget_terms', [])
bud = summarize_budget(cbc_path, terms, kstpkper=(0,0)) if terms else {}
print('\nBudget summary (sum over cells, SP0/TS0):')
for k,v in bud.items():
    print(f'  {k:>16}: {v}')


## 8. Analytical check (required)
Compare your modeled drawdown to a simple analytical expectation at a chosen observation radius; document assumptions.

In [ ]:
ap = cfg['analysis_plan']
method = ap.get('method', 'Thiem_confined')
Q_abs = float(ap.get('Q_abs_m3d', 0.0))
r_obs = float(ap.get('r_obs_m', 1.0))
r_ref = float(ap.get('r_ref_m', 0.1))
if method == 'Thiem_confined':
    T = float(ap.get('T_m2d', 1.0))
    s_thiem = thiem_confined_drawdown(Q_abs, T, r_obs, r_ref)
    print('Thiem confined drawdown (m):', s_thiem)
else:
    # Simple Dupuit-like approximation if desired
    K = float(ap.get('K_mpd', 10.0))
    b = float(ap.get('b_m', 50.0))
    T = K * b
    s_thiem = thiem_confined_drawdown(Q_abs, T, r_obs, r_ref)
    print('Dupuit (using T=K*b) drawdown (m):', s_thiem)

print('Reminder: choose an observation point near r_obs for comparison and discuss discrepancies.')


## 9. Interpretation (write here)
**Required bullets:**
- Brief restatement of the question and key assumptions
- Head/drawdown patterns and physical interpretation
- River/well budget changes and stream depletion fraction (if applicable)
- Comparison to analytical expectation (agree? why/why not?)
- One sensitivity (e.g., halve/double K or river conductance) — describe the effect qualitatively/quantitatively


## 10. Checklist for submission
- [ ] `case_student.yaml` filled (no TODOs)
- [ ] Notebook runs top-to-bottom without errors
- [ ] Required figures: head map, drawdown map
- [ ] Observation heads table
- [ ] Budget summary table
- [ ] Analytical check with clearly stated parameters
- [ ] 1–2 paragraph interpretation & sensitivity notes
